# *Salix nigra* SLR: Alignments
Brian J. Sanderson

Last updated: 27 Mar 2020

In [1]:
# Libraries used in this notebook (R 3.6.2)
suppressMessages(library(tidyverse))

## Preparing raw reads for alignments

These steps implement the Broad Institute best practices for data pre-processing (current as of 2018-06-29) found [here](https://software.broadinstitute.org/gatk/best-practices/workflow?id=11165). Note that we are not doing base score recalibration because we lack the necessary "gold standard" SNP information in our species.

## Convert raw FASTQ to uBAM files with appropriate read group info

It's possible to work with raw FASTQ reads for alignment, but they lack metadata. The Broad Institute (makers of GATK) recommend converting FASTQ files into unaligned BAM files, which allows you to properly specify the relevant metadata. The following code does this. 

*Note*: this code chunk, and all of the ones that follow up until the section [Create a scatter file for variant detection](#Create-a-scatter-file-for-variant-detection) below are SLURM submission scripts that were run on the HPCC at Texas Tech University. Some modification is likely necessary to run on different HPC environments.

The file ```allnames-short.txt``` referenced below is just a text file with the names of each library, one per line, which allows for the assignment of a prefix variable for the fastq files and the resulting BAM file.

```bash
# HPC submisison script

#!/bin/sh
#$ -V
#$ -cwd
#$ -t 1-96:1
#$ -S /bin/bash
#$ -N snigra-alignment-qc
#$ -q omni
#$ -o log/00/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e log/00/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -pe sm 1
#$ -P quanah

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
prefix=`sed -n "$prefixNumP" allnames-short.txt`

gatk FastqToSam --java-options "-Xmx8G" \
    --FASTQ reads/"$prefix"_R1_001.fastq \
    --FASTQ2 reads/"$prefix"_R2_001.fastq \
    --OUTPUT uBAM/SN"$prefix".unmapped.bam \
    --READ_GROUP_NAME Salix_nigra_SLR \
    --SAMPLE_NAME "$prefix" \
    --LIBRARY_NAME NEBNext-"$prefix" \
    --PLATFORM_UNIT HT23LBBXX.6.1101 \
    --PLATFORM illumina \
    --SEQUENCING_CENTER OMRF \
    --RUN_DATE 2018-03-12T08:55:00+0500
```

## Mapped the reads to the *Salix purpurea* PacBio v5 assembly

Here we map the reads for each library onto the *Salix purpurea* PacBio assembly

```bash
# HPC submisison script
#!/bin/sh
#$ -V
#$ -cwd
#$ -S /bin/bash
#$ -t 1-96:1
#$ -N snigra-alignment-qc
#$ -q omni
#$ -e log/00/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -o log/00/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -pe sm 16
#$ -P quanah

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
prefix=`sed -n "$prefixNumP" allnames.txt`

gatk --java-options "-Dsamjdk.compression_level=5 -Xms3000m" SamToFastq \
    --INPUT uBAM/"$prefix".unmapped.bam \
    --FASTQ /dev/stdout \
    --INTERLEAVE true \
    --NON_PF true \
    | \
    bwa mem -K 100000000 -p -v 3 -t 16 \
    -Y Salix_purpurea_var_94006.mainGenome-noZ.fasta \
    /dev/stdin - \
    | \
    samtools view -1 - > aBAM/"$prefix".aligned.bam
```

## Merge aligned and unaligned BAMs

Here we merge the aligned BAMs we created with the previous step with the original unaligned BAM files. This is useful for identifying erroneous reads that resulted from optical duplicate clusters in the next step.

```bash
# HPC submisison script

#!/bin/sh
#$ -V
#$ -cwd
#$ -t 1-96:1
#$ -S /bin/bash
#$ -N snigra-alignment-qc
#$ -q omni
#$ -o log/00/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e log/00/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -pe sm 1
#$ -P quanah

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
prefix=`sed -n "$prefixNumP" allnames.txt`

gatk  --java-options "-Dsamjdk.compression_level=5 -Xms3000m" MergeBamAlignment \
    --VALIDATION_STRINGENCY SILENT \
    --EXPECTED_ORIENTATIONS FR \
    --ATTRIBUTES_TO_RETAIN X0 \
    --ALIGNED_BAM aBAM/"$prefix".aligned.bam \
    --UNMAPPED_BAM uBAM/"$prefix".unmapped.bam \
    --OUTPUT mBAM/"$prefix".merged.bam \
    --REFERENCE_SEQUENCE Salix_purpurea_var_94006.mainGenome-noZ.fasta \
    --PAIRED_RUN true \
    --SORT_ORDER 'unsorted' \
    --IS_BISULFITE_SEQUENCE false \
    --ALIGNED_READS_ONLY false \
    --CLIP_ADAPTERS false \
    --MAX_RECORDS_IN_RAM 2000000 \
    --ADD_MATE_CIGAR true \
    --MAX_INSERTIONS_OR_DELETIONS -1 \
    --PRIMARY_ALIGNMENT_STRATEGY MostDistant \
    --PROGRAM_RECORD_ID 'bwamem' \
    --PROGRAM_GROUP_VERSION '0.7.17-r1188' \
    --PROGRAM_GROUP_COMMAND_LINE 'bwa mem -K 100000000 -p -v 3 -t 16 \
        -Y Salix_purpurea_var_94006.mainGenome-noZ.fasta /dev/stdin -' \
    --PROGRAM_GROUP_NAME 'bwamem' \
    --UNMAPPED_READ_STRATEGY COPY_TO_TAG \
    --ALIGNER_PROPER_PAIR_FLAGS true \
    --UNMAP_CONTAMINANT_READS true
```

## Sort and fix tags

Here we sort the alignments by coordinates, and then fix Nm and Uq tags.

```bash
# HPC submisison script

#!/bin/sh
#$ -V
#$ -cwd
#$ -t 1-96:1
#$ -S /bin/bash
#$ -N snigra-alignment-qc
#$ -q omni
#$ -o log/00/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e log/00/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -pe sm 1
#$ -P quanah

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
prefix=`sed -n "$prefixNumP" allnames.txt`

gatk --java-options "-Dsamjdk.compression_level=5 -Xms4000m" SortSam \
    --INPUT mBAM/"$prefix".merged.bam \
    --OUTPUT /dev/stdout \
    --SORT_ORDER 'coordinate' \
    --CREATE_INDEX false \
    --CREATE_MD5_FILE false \
    | \
    gatk --java-options "-Dsamjdk.compression_level=5 -Xms500m" SetNmMdAndUqTags \
    --INPUT /dev/stdin \
    --OUTPUT sBAM/"$prefix".sorted.bam \
    --CREATE_INDEX true \
    --CREATE_MD5_FILE true \
    --REFERENCE_SEQUENCE Salix_purpurea_var_94006.mainGenome-noZ.fasta
```

## Identify duplicated reads using picard

Here we identify and remove reads that are erronous results of optical duplicates.

```bash
# HPC submisison script

#!/bin/sh
#$ -V
#$ -cwd
#$ -t 1-96:1
#$ -S /bin/bash
#$ -N snigra-alignment-qc
#$ -q omni
#$ -o log/00/$JOB_NAME.o$JOB_ID.$TASK_ID
#$ -e log/00/$JOB_NAME.e$JOB_ID.$TASK_ID
#$ -pe sm 1
#$ -P quanah

prefixNum="$SGE_TASK_ID"
prefixNumP="$prefixNum"p
prefix=`sed -n "$prefixNumP" allnames.txt`

gatk --java-options "-Dsamjdk.compression_level=5 -Xms4000m" MarkDuplicates \
    --INPUT sBAM/"$prefix".sorted.bam \
    --OUTPUT dBAM/"$prefix".dedup.bam \
    --METRICS_FILE dBAM/"$prefix".dedup.metrics \
    --VALIDATION_STRINGENCY SILENT \
    --OPTICAL_DUPLICATE_PIXEL_DISTANCE 2500 \
    --ASSUME_SORT_ORDER 'coordinate' \
    --CREATE_INDEX true \
    --CREATE_MD5_FILE true
```

## Create a scatter file for variant detection

The following Python code will prepare a set of 100,000 bp intervals across all chromosomes and scaffolds in the *S. purpurea* genome. This will be used in later analyses to parallelize the variant detection

```python
# Python code
import csv
from Bio import SeqIO

with open("Salix_purpurea_var_94006.mainGenome-noZ.fasta", 'r') as inputSeqFile:
    SeqDict = SeqIO.to_dict(SeqIO.parse(inputSeqFile, 'fasta'))

   
with open("salixIntervals.list", 'wt', newline='\n') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for chrom in SeqDict:
        end = len(SeqDict[chrom].seq)
        i = 1
        j = 100000
        if end < j:
            writer.writerow([chrom + ":" + str(i) + "-" + str(end)])
            next
        else:
            while end > j:
                writer.writerow([chrom + ":" + str(i) + "-" + str(j)])
                if end > j + 100000:
                    i = j + 1
                    j = j + 100000
                    next
                elif end < j + 100000:
                    i = j + 1
                    writer.writerow([chrom + ":" + str(i) + "-" + str(end)])
                    j = j + 100000
                    break
```

## Quantify depth across the target loci

Finally, we use bedtools to quantify the read depth in our analysis-ready BAM files. First, align the bait sequences (```salix_bait_targets.fasta```) to the reference genome

```bash
bwa mem -K 100000000 -v 3 -t 16 -Y Salix_purpurea_var_94006.mainGenome-noZ.fasta salixSLRtargets.fasta | samtools view -1 - > salix_bait_targets.bam
```

Next, convert the bait targets and each of our analysis-ready BAM files into BED files. This uses GNU parallel to run this over 20 BAMs at a time.

```bash
bedtools bamtobed -i salix_bait_targets.bam > salix_bait_targets.bed
parallel -j 20 --gnu --eta "bedtools bamtobed -i {}.dedup.bam > {}.bed" :::: allnames_short.txt
```

Finally, use bedtools intersect to quantify the read depth across the targeted loci:

```bash
parallel -j 20 --gnu --eta "bedtools intersect -c -a salix_bait_targets.bed \
                            -b {}.bed > {}.depth" :::: allnames_short.txt
```

## Summarize read depth across targeted loci in R

Create a list of the depth files in the current directory

In [2]:
files <- list.files(".", pattern = "*.depth")
big.df <- vector('list', length(files))

Create a data frame that contains the depths for each library

In [3]:
suppressMessages(for (i in 1:length(files)){
  ## Read table
  name_vec <- str_match(files[i], "(.*)\\..*$")
  tmp<-read_delim(name_vec[1], delim = "\t", 
                                 col_names = c("chr", "start", "stop", "gene", 
                                               "score", "dir", "count")) %>% 
                        mutate(., library = name_vec[2])
  big.df[[i]] <- tmp

})
bam_df <- do.call('rbind', big.df)

### Distribution of read counts across all reads

In [4]:
group_by(bam_df) %>%
  summarise(., "5%" = quantile(count, 0.05),
               "25%" = quantile(count, 0.25),
               "50%" = quantile(count, 0.5),
               "75%" = quantile(count, 0.75),
               "99%" = quantile(count, 0.99),
               "MAX" = max(count), 
               "mean" = exp(mean(log(count + 1))) - 1,
               "sd" = exp(sd(log(count + 1))) - 1,
               "se" = sd / sqrt(n()))

5%,25%,50%,75%,99%,MAX,mean,sd,se
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,20,49,112,617,15834,44.679,2.683072,0.002170975


### Distribution of read counts for each library

In [5]:
group_by(bam_df, library) %>%
  summarise(., "5%" = quantile(count, 0.05),
               "25%" = quantile(count, 0.25),
               "50%" = quantile(count, 0.5),
               "75%" = quantile(count, 0.75),
               "99%" = quantile(count, 0.99),
               "MAX" = max(count), 
               "mean" = exp(mean(log(count + 1))) - 1,
               "sd" = exp(sd(log(count + 1))) - 1)

library,5%,25%,50%,75%,99%,MAX,mean,sd
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SN502F,4,22,51,110,463.0,7038,45.52219,2.456412
SN503M,2,11,26,57,256.0,1221,24.10149,2.295431
SN504F,2,13,29,63,278.8,1830,26.53666,2.265926
SN505F,8,39,90,196,881.0,7883,80.57315,2.601541
SN506F,5,26,60,130,551.8,3304,53.72216,2.473303
SN507M,8,37,84,185,826.0,5701,76.04291,2.563148
SN508F,10,46,106,230,1020.8,5466,94.88032,2.582878
SN509F,4,19,44,97,443.8,3289,40.34973,2.447437
SN510F,6,32,74,164,710.0,3184,66.55650,2.573792
